# GUC Clustering Project


**Objective:**
The objective of this project teach students how to apply clustering to real data sets

The projects aims to teach student:

- Which clustering approach to use
- Compare between Kmeans, Hierarchal, DBScan, and Gaussian Mixtures
- How to tune the parameters of each data approach
- What is the effect of different distance functions (optional)
- How to evaluate clustering approachs
- How to display the output
- What is the effect of normalizing the data

Students in this project will use ready-made functions from Sklearn, plotnine, numpy and pandas


In [ ]:
# if plotnine is not installed in Jupyter then use the following command to install it
# use at anaconda prompt with it being run as admin
# one time only then it is never needed again
# conda install -c conda-forge plotnine
# done


Running this project require the following imports


In [ ]:
from plotnine import *
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_blobs
import sklearn.preprocessing as prep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# StandardScaler is a function to normalize the data
# You may also check MinMaxScaler and MaxAbsScaler
#from sklearn.preprocessing import StandardScaler


# hierarchical clustering


# for cdist in distortions graph

%matplotlib inline


In [ ]:
# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(X, km=[], num_clusters=0):
    alpha = 0.5  # color opaque
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:, 0], X[:, 1], c="r", alpha=alpha, s=s)
    else:
        colors = list(map(lambda x: "#3b4cc0" if x == 1 else "#b40426", km.labels_))
        for i in range(num_clusters):
            plt.scatter(
                X[km.labels_ == i, 0], X[km.labels_ == i, 1], c=colors, alpha=alpha, s=s
            )
            plt.scatter(
                km.cluster_centers_[i][0],
                km.cluster_centers_[i][1],
                c=colors,
                marker="o",
                s=100,
            )

## Multi Blob Data Set

- The Data Set generated below has 6 cluster with varying number of users and varing densities
- Cluster the data set below using


In [ ]:
plt.rcParams["figure.figsize"] = [8, 8]
sns.set_style("whitegrid")
sns.set_context("talk")

n_bins = 6
centers = [(-3, -3), (0, 0), (5, 2.5), (-1, 4), (4, 6), (9, 7)]
Multi_blob_Data, y = make_blobs(
    n_samples=[100, 150, 300, 400, 300, 200],
    n_features=2,
    cluster_std=[1.3, 0.6, 1.2, 1.7, 0.9, 1.7],
    centers=centers,
    shuffle=False,
    random_state=42,
)
display_cluster(Multi_blob_Data)

### Kmeans

- Use Kmeans with different values of K to cluster the above data
- Display the outcome of each value of K
- Plot distortion function versus K and choose the approriate value of k
- Plot the silhouette_score versus K and use it to choose the best K
- Store the silhouette_score for the best K for later comparison with other clustering techniques.


#### plot and display k means with 10 k values


In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(30, 12))

i, j = 0, 0
clust_range = range(1, 11)
for num_clust in clust_range:

    if i == 3:
        break
    if j == 5:
        j = 0
        i += 1

    # get K means
    kmeans = KMeans(n_clusters=num_clust)
    kmeans.fit(Multi_blob_Data)
    y_kmeans = kmeans.predict(Multi_blob_Data)

    ax[i][j].scatter(
        Multi_blob_Data[:, 0], Multi_blob_Data[:, 1], c=y_kmeans, s=20, cmap="hsv"
    )
    # draw centers
    centers = kmeans.cluster_centers_
    ax[i][j].scatter(centers[:, 0], centers[:, 1], c="black", s=100, alpha=0.5)

    # did not use display cluster as color list only has the seven single letter built in cluster
    # the method I used here is almost the same but working

    # set title for each graph

    ax[i][j].set_title(f"K-means with {num_clust} clusters")
    ttl = ax[i][j].title
    ttl.set_position([0.5, 2])

    j += 1

fig.suptitle("K-means for multi blob data")
fig.tight_layout()

#### Distortion function vs K


In [ ]:
i, j = 0, 0
inertia = []
distortions = []
silhouette_avg = []
for num_clust in clust_range:

    # get K means
    kmeans = KMeans(n_clusters=num_clust)
    kmeans.fit(Multi_blob_Data)

    cluster_labels = kmeans.labels_

    inertia.append(kmeans.inertia_)
    distortions.append(
        sum(
            np.min(cdist(Multi_blob_Data, kmeans.cluster_centers_, "euclidean"), axis=1)
        )
        / Multi_blob_Data.shape[0]
    )

    # silhouette score
    if num_clust != 1:
        silhouette_avg.append(silhouette_score(Multi_blob_Data, cluster_labels))

plt.plot(clust_range, inertia, "bx-")
plt.xlabel("k")
plt.ylabel("Sum_of_squared_distances")
plt.title("Elbow Method For Optimal k")
plt.show()

#### Find silhouette score


In [ ]:
# silhouette score
plt.plot(range(2, 11), silhouette_avg, "bx-")
plt.xlabel("Values of K")
plt.ylabel("Silhouette score")
plt.title("Silhouette analysis For Optimal k")
plt.show()

# +2 because range starts from 2 as we removed one cluster as it does not work on silhouette score
print("best K is", pd.Series(silhouette_avg).idxmax() + 2)


### Hierarchal Clustering

- Use AgglomerativeClustering function to to cluster the above data
- In the AgglomerativeClustering change the following parameters
  - Affinity (use euclidean, manhattan and cosine)
  - Linkage( use average and single )
  - Distance_threshold (try different)
- For each of these trials plot the Dendograph , calculate the silhouette_score and display the resulting clusters
- Find the set of paramters that would find result in the best silhouette_score and store this score for later comparison with other clustering techniques.
- Record your observation


In [ ]:
# dendrogram to help with number of clusters
fig, ax = plt.subplots(1, 3, figsize=(30, 12))

# average
ax[0].set_title("Dendrograms for average")
dend_avg = hierarchy.dendrogram(
    hierarchy.linkage(Multi_blob_Data, method="average"), ax=ax[0]
)
ax[0].axhline(y=5, color="r", linestyle="--")
# single
ax[1].set_title("Dendrograms for single")
dend_single = hierarchy.dendrogram(
    hierarchy.linkage(Multi_blob_Data, method="single"), ax=ax[1]
)
ax[1].axhline(y=5, color="r", linestyle="--")
# ward
ax[2].set_title("Dendrograms for ward")
dend_single = hierarchy.dendrogram(
    hierarchy.linkage(Multi_blob_Data, method="ward"), ax=ax[2]
)
ax[2].axhline(y=100, color="r", linestyle="--")

#### Takeaways based on dendrogram

- Average is better than single so it will be used
- After using the dendrogram for average and drawing the threshold line clusters number will be 5
- this is a general guideline we will use distance threshold in this example


In [ ]:
hierarchal_range = range(1, 25)
affinity = ["euclidean", "manhattan", "cosine"]
distance_threshold = [1, 2, 3, 4, 5, 6, 7, 8]
fig, axes = plt.subplots(3, 8, figsize=(40, 20))
i, j = 0, 0


for aggloClusterGraph in hierarchal_range:

    if i == 3:
        break
    if j == 8:
        j = 0
        i += 1

    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        affinity=affinity[i],
        linkage="average",
        distance_threshold=distance_threshold[j],
    )
    # clustering_model.fit(Multi_blob_Data)
    HClusteringY = clustering_model.fit_predict(Multi_blob_Data)

    labels = clustering_model.labels_

    axes[i][j].scatter(
        Multi_blob_Data[:, 0], Multi_blob_Data[:, 1], c=HClusteringY, cmap="tab10"
    )

    if np.any(HClusteringY.T):
        # get centroids
        clf = NearestCentroid()
        # print(HClusteringY)
        clf.fit(Multi_blob_Data, HClusteringY)
        axes[i][j].scatter(
            clf.centroids_[:, 0],
            clf.centroids_[:, 1],
            c="black",
            s=100,
            alpha=0.5,
            marker="x",
        )

    # set title for each graph
    axes[i][j].set_title(
        f"Affinity {affinity[i]} and Distance threshold {distance_threshold[j]}"
    )
    ttl = axes[i][j].title
    ttl.set_position([0.5, 2])

    j += 1


fig.suptitle("Agglomerative clustering for multi blob data")
fig.tight_layout()

Some Takeaways

- single is terrible all the results are mostly 1 cluster with some outliers
- cosine affinity is not suitable at all for this dataset
- distance threshold from 4 to 6 should only be considered as out of this range clusters are too many or too few

All the results for single and cosine with average linkage was combined in the code below as they are unimportant they are only for reference
we will run the same tests again but using the new range for better results


In [ ]:
# same code as above but with single and cosine average
hierarchal_range = range(1, 29)
affinity = ["euclidean", "manhattan", "cosine", "cosine"]
linkage = ["single", "single", "single", "average"]
distance_threshold = [1, 2, 3, 4, 5, 6, 7, 8]
fig, axes = plt.subplots(4, 7, figsize=(45, 25))
i, j = 0, 0


for aggloClusterGraph in hierarchal_range:

    if i == 4:
        break
    if j == 7:
        j = 0
        i += 1

    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        affinity=affinity[i],
        linkage=linkage[i],
        distance_threshold=distance_threshold[j],
    )
    HClusteringY = clustering_model.fit_predict(Multi_blob_Data)
    # labels = clustering_model.labels_

    axes[i][j].scatter(
        Multi_blob_Data[:, 0], Multi_blob_Data[:, 1], c=HClusteringY, cmap="bwr"
    )

    if np.any(HClusteringY.T):
        # get centroids
        clf = NearestCentroid()
        # print(HClusteringY)
        clf.fit(Multi_blob_Data, HClusteringY)
        axes[i][j].scatter(
            clf.centroids_[:, 0],
            clf.centroids_[:, 1],
            c="black",
            s=100,
            alpha=0.5,
            marker="x",
        )

    # set title for each graph
    axes[i][j].set_title(
        f"affinity {affinity[i]} and distance threshold {distance_threshold[j]}"
    )
    if i == 3:
        axes[i][j].set_title(
            f"affinity {affinity[i]} and distance threshold {distance_threshold[j]} (average)"
        )
    ttl = axes[i][j].title
    ttl.set_position([0.5, 2])

    j += 1


fig.suptitle("Agglomerative clustering for multi blob data using single linkage")
fig.tight_layout()

#### Try Agglomerative clustering with new range (4 to 6)

manhattan from 6 to 8 was tried as well


In [ ]:
hierarchal_range = range(1, 22)
affinity = ["euclidean", "manhattan", "manhattan"]
distance_threshold = [4, 4.5, 5, 5.25, 5.5, 5.75, 6]
distance_threshold_manhattan = [6.25, 6.5, 6.75, 7, 7.25, 7.5, 8]
fig, axes = plt.subplots(3, 7, figsize=(40, 15))


silhouette_avg = []
i, j = 0, 0
for aggloClusterGraph in hierarchal_range:

    if i == 3:
        break
    if j == 7:
        j = 0
        i += 1
    if i == 2:
        distance_threshold = distance_threshold_manhattan
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        affinity=affinity[i],
        linkage="average",
        distance_threshold=distance_threshold[j],
    )
    # clustering_model.fit(Multi_blob_Data)
    HClusteringY = clustering_model.fit_predict(Multi_blob_Data)
    labels = clustering_model.labels_

    axes[i][j].scatter(
        Multi_blob_Data[:, 0],
        Multi_blob_Data[:, 1],
        c=HClusteringY,
        cmap="gist_rainbow",
    )

    # to get centroids T returns a array if all is zero then only one cluster is present
    # which means no getting centroids as having one cluster only causes an error in the function
    if np.any(HClusteringY.T):
        # get centroids
        clf = NearestCentroid()
        # print(HClusteringY)
        clf.fit(Multi_blob_Data, HClusteringY)
        axes[i][j].scatter(
            clf.centroids_[:, 0],
            clf.centroids_[:, 1],
            c="black",
            s=100,
            alpha=0.5,
            marker="x",
        )

    # silhouette score
    silhouette_avg.append(silhouette_score(Multi_blob_Data, labels))

    # set title for each graph
    axes[i][j].set_title(
        f"Affinity {affinity[i]} and Distance threshold {distance_threshold[j]}"
    )
    ttl = axes[i][j].title
    ttl.set_position([0.5, 2])

    j += 1


fig.suptitle(
    "Second Agglomerative clustering for multi blob data with a confined range of 4 to 6"
)
fig.tight_layout()

# used in next step
distance_threshold = [4, 4.5, 5, 5.25, 5.5, 5.75, 6]

In [ ]:
# silhouette score
plt.plot(range(0, 7), silhouette_avg[0:7], "bx-", label="euclidean 4 to 6", color="r")
plt.plot(range(7, 14), silhouette_avg[7:14], "bx-", label="manhattan 4 to 6", color="b")
plt.plot(
    range(14, 21), silhouette_avg[14:21], "bx-", label="manhattan 6 to 8", color="g"
)
plt.xlabel("Values of distance and affinity")
plt.ylabel("Silhouette score")
plt.title("Silhouette analysis For optimal Agglomerative clustering")
plt.legend(loc="best")
plt.show()

optimal_index = pd.Series(silhouette_avg).idxmax()
opt_div = optimal_index // 7
affinity = ""

if optimal_index < 14:
    distance_thresh = distance_threshold[optimal_index % 7]
else:
    distance_thresh = distance_threshold_manhattan[optimal_index % 7]

if optimal_index < 7:
    affinity = "euclidean"
else:
    affinity = "manhattan"


print(
    f"best Agglomerative clustering is at index {optimal_index} which is {affinity} affinity with distance threshold of {distance_thresh}"
)
print(distance_threshold[1])

#### Summary till now:

#### Kmeans

best K is at 6

#### Agglomerative clustering

best parameters are euclidean affinity with distance threshold of 4.5


### DBScan

- Use DBScan function to to cluster the above data
- In the DBscan change the following parameters
  - EPS (from 0.1 to 3)
  - Min_samples (from 5 to 25)
- Plot the silhouette_score versus the variation in the EPS and the min_samples
- Plot the resulting Clusters in this case
- Find the set of paramters that would find result in the best silhouette_score and store this score for later comparison with other clustering techniques.
- Record your observations and comments


Compute data proximity from each other using Nearest Neighbours


In [ ]:
neighb = NearestNeighbors(
    n_neighbors=5
)  # creating an object of the NearestNeighbors class
nbrs = neighb.fit(Multi_blob_Data)  # fitting the data to the object
distances, indices = nbrs.kneighbors(Multi_blob_Data)  # finding the nearest neighbours

distances = np.sort(distances, axis=0)  # sorting the distances
distances = distances[:, 1]  # taking the second column of the sorted distances
plt.rcParams["figure.figsize"] = (5, 3)  # setting the figure size
plt.plot(distances)  # plotting the distances
plt.show()  # showing the plot

after plotting the main graph we find that the maximum curvature is at eps = 0.5 so this should be the best eps to use


In [ ]:
df_Multi_blob = Multi_blob_Data

# clustering function
def DB_Clustering(eps, num_sample):
    DBSCAN_Clustering_model = DBSCAN(eps=eps, min_samples=num_sample).fit(
        Multi_blob_Data
    )
    return DBSCAN_Clustering_model.labels_


def DBSCAN_Subplots(nrows, ncols, inverse, eps_max, min_samples_max):
    # create subplots and ranges
    num_Graphs = nrows * ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 12, nrows * 10))
    eps = np.linspace(0.1, eps_max, num=num_Graphs)
    if inverse:
        min_samples = np.linspace(min_samples_max, 5, num=num_Graphs)
    else:
        min_samples = np.linspace(5, min_samples_max, num=num_Graphs)
    DBSCAN_Range = range(0, num_Graphs)

    silhouette_avg = []
    i, j = 0, 0
    for DBSCAN_Graph in DBSCAN_Range:
        if i == nrows:
            break
        if j == ncols:
            j = 0
            i += 1

        DBSCAN_label = DB_Clustering(eps[DBSCAN_Graph], min_samples[DBSCAN_Graph])
        axes[i][j].scatter(
            x=df_Multi_blob[:, 0],
            y=df_Multi_blob[:, 1],
            c=DBSCAN_label,
            cmap="Accent",
            s=50,
        )

        # silhouette score
        # if to check if the label array is all equal, if all equal then only one cluster is present which throws an error in silhouette function
        if len(np.unique(DBSCAN_label)) > 1:
            silhouette_avg.append(
                {
                    "value": silhouette_score(Multi_blob_Data, DBSCAN_label),
                    "eps": eps[DBSCAN_Graph],
                    "min_samples": min_samples[DBSCAN_Graph],
                }
            )

            # print(silhouette_avg[-1],DBSCAN_Graph)

        # set title for each graph
        axes[i][j].set_title(
            f"DBSCAN with eps of {round(eps[DBSCAN_Graph],3)} and minimum samples of {round(min_samples[DBSCAN_Graph],3)}"
        )
        ttl = axes[i][j].title
        ttl.set_position([0.5, 2])

        j += 1
    fig.tight_layout()
    fig.suptitle(
        f"DBSCAN clustering for multi blob data with using {num_Graphs} graphs and eps from 0.1 to {eps_max} and minimum samples from 5 to {min_samples_max}",
        y=1.02,
    )
    return silhouette_avg


# DBSCAN_Subplots(4,4,False)
silhouette_avg = DBSCAN_Subplots(5, 5, True, 3, 25)
silhouette_avg2 = DBSCAN_Subplots(5, 5, False, 3, 25)

# for testing
# plt.scatter(x=df_Multi_blob[:,0],y=df_Multi_blob[:,1],c=DB_Clustering(0.4,5),cmap="hsv",s=50)

In [ ]:
silhouette_avg3 = DBSCAN_Subplots(5, 5, True, 2, 25)
silhouette_avg4 = DBSCAN_Subplots(5, 5, False, 2, 25)


After performing the first dbscan wave it is time to narrow down the parameters

the new eps range is 0.1 to 1.1

the new min samples range is the same


In [ ]:
silhouette_avg5 = DBSCAN_Subplots(5, 5, True, 1.1, 25)
silhouette_avg6 = DBSCAN_Subplots(5, 5, False, 1.1, 25)


In [ ]:
def get_silhouette_avg(
    silhouette_avg,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
):
    for idx, dicts in enumerate(silhouette_avg):
        silhouette_avg_values.append(silhouette_avg[idx]["value"])
        silhouette_avg_eps.append(silhouette_avg[idx]["eps"])
        silhouette_avg_min_samples.append(silhouette_avg[idx]["min_samples"])
        all_silhouette_avg.append(silhouette_avg[idx])


silhouette_avg_values = []
silhouette_avg_eps = []
silhouette_avg_min_samples = []
all_silhouette_avg = []

get_silhouette_avg(
    silhouette_avg,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
)
get_silhouette_avg(
    silhouette_avg2,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
)
get_silhouette_avg(
    silhouette_avg3,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
)
get_silhouette_avg(
    silhouette_avg4,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
)
get_silhouette_avg(
    silhouette_avg5,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
)
get_silhouette_avg(
    silhouette_avg6,
    silhouette_avg_values,
    silhouette_avg_eps,
    silhouette_avg_min_samples,
    all_silhouette_avg,
)

# print(len(silhouette_avg_values),len(silhouette_avg_eps),len(silhouette_avg_min_samples))
optimal_index = pd.Series(silhouette_avg_values).idxmax()
optimal_eps = silhouette_avg_eps[optimal_index]
optimal_min_samples = silhouette_avg_min_samples[optimal_index]

print(f"best eps is {optimal_eps} ,best min_samples is {optimal_min_samples}")

# silhouette score
plt.plot(range(1, len(all_silhouette_avg) + 1), silhouette_avg_values, color="r")
plt.xlabel("index of dbscan")
plt.ylabel("Silhouette score")
plt.title("Silhouette analysis For optimal Agglomerative clustering")


plt.show()

### Gaussian Mixture

- Use GaussianMixture function to cluster the above data
- In GMM change the covariance_type and check the difference in the resulting proabability fit
- Use a 2D contour plot to plot the resulting distribution (the components of the GMM) as well as the total Gaussian mixture


In [ ]:
# Create a list of n_components values
n_components = list(range(1, 21))

fig, axes = plt.subplots(5, 4, figsize=(18, 22))

# Loop over the n_components values
for n, ax in zip(n_components, axes.ravel()):
    # Create a GaussianMixture object with n components and fit model
    gm = GaussianMixture(n_components=n).fit(Multi_blob_Data)
    # Predict the labels for each data point
    labels = gm.predict(Multi_blob_Data)
    # Plot the data points with different colors according to their labels
    ax.scatter(
        Multi_blob_Data[:, 0], Multi_blob_Data[:, 1], c=labels, cmap="gist_rainbow"
    )

    # Add a title with the number of components
    ax.set_title(f"GMM with {n} components")
# Adjust the spacing between subplots
fig.tight_layout()
# Show the figure
plt.show()

#### Gaussian mixture with different covariance types and with 2d contour plot


In [ ]:
X = Multi_blob_Data

# Define the range of components and covariance types
n_components_range = range(1, 21)
covariance_types = ["full", "tied", "diag", "spherical"]

# Loop over the covariance types
for i, cov_type in enumerate(covariance_types):
    # Create a figure with 20 subplots (4 rows and 5 columns) for each covariance type
    fig, axes = plt.subplots(4, 5, figsize=(35, 28))
    # Loop over the range of components
    for j, n_components in enumerate(n_components_range):
        # Create a GMM object with the given parameters
        gmm = GaussianMixture(
            n_components=n_components, covariance_type=cov_type, random_state=0
        )
        # Fit the GMM to the data
        gmm.fit(X)
        # Predict the labels for each data point
        labels = gmm.predict(X)
        # Compute the log probability density of each data point under the model
        log_prob = gmm.score_samples(X)

        # Plot the data points with different colors for each label on the corresponding subplot
        axes[j // 5][j % 5].scatter(X[:, 0], X[:, 1], c=labels, s=10, cmap="hsv")

        # Set up a grid of points for plotting contours
        x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
        y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
        xx, yy = np.meshgrid(
            np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100)
        )
        xy = np.column_stack([xx.ravel(), yy.ravel()])

        # Compute and plot contours of probability density on top of data points
        z_log_prob = gmm.score_samples(xy).reshape(xx.shape)
        axes[j // 5][j % 5].contour(xx, yy, z_log_prob)

        axes[j // 5][j % 5].set_title(
            f"GMM with {cov_type} covariance and {n_components} components",
            fontsize=16,
            y=1.05,
        )
        axes[j // 5][j % 5].set_xlabel("x")
        axes[j // 5][j % 5].set_ylabel("y")
        ttl = axes[j // 5][j % 5].title
        ttl.set_position([0.5, 1])

# Adjust the spacing between subplots
plt.tight_layout()
# Show the figure
plt.show()

## iris data set

The iris data set is test data set that is part of the Sklearn module
which contains 150 records each with 4 features. All the features are represented by real numbers

The data represents three classes


In [ ]:
from sklearn.datasets import load_iris

iris_data = load_iris()
iris_data.target[[10, 25, 50]]
# array([0, 0, 1])
list(iris_data.target_names)
["setosa", "versicolor", "virginica"]

### K means clustering on iris dataset


In [ ]:
iris = load_iris()
X = iris.data


def kmeans_iris(x, y):
    fig, axs = plt.subplots(2, 5, figsize=(20, 8))
    axs = axs.ravel()

    sil_scores = []
    distortion = []

    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(X)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_
        distortion.append(kmeans.inertia_)

        if i > 1:
            sil_score = silhouette_score(X, labels)
            sil_scores.append(sil_score)
            axs[i - 1].set_title(f"k={i}, silhouette={sil_score:.2f}", y=1.01)
        else:
            axs[i - 1].set_title(f"k={i}", y=1.01)

        # plot cluster and centers
        axs[i - 1].scatter(X[:, x], X[:, y], c=labels, cmap="gist_rainbow")
        axs[i - 1].scatter(
            centers[:, x], centers[:, y], marker="o", s=100, color="black"
        )

        if x == 0 and y == 1:
            axs[i - 1].set_xlabel("sepal_length")
            axs[i - 1].set_ylabel("sepal_width")
        else:
            axs[i - 1].set_xlabel("petal_length")
            axs[i - 1].set_ylabel("petal_width")

    fig.tight_layout()
    return distortion, sil_scores


distortion, sil_scores = kmeans_iris(0, 1)
kmeans_iris(2, 3)


# create elbow graph
plt.figure()
plt.plot(range(1, 11), distortion)
plt.title("The elbow method")
plt.xlabel("Number of clusters")
plt.ylabel("SUm of squares distance")  # within cluster sum of squares
plt.show()

# create and plot sil graph
plt.figure()
plt.plot(range(2, 11), sil_scores)
plt.xlabel("k")
plt.ylabel("Silhouette Score")
plt.show()

According to the elbow method k=3 is the best solution


### Hierarchal Clustering


In [ ]:
iris = load_iris()
X = iris.data


def aggro_hierarchal_iris(dist_min, dist_max):
    affinities = ["euclidean", "manhattan", "cosine"]
    linkages = ["average", "single"]
    distance_thresholds = np.linspace(dist_min, dist_max, num=16)

    # fig, axs = plt.subplots(len(affinities) * len(linkages), len(distance_thresholds), figsize=(50, 25))

    best_score = -1
    best_params = []
    sil_score = []

    for i, affinity in enumerate(affinities):
        for j, linkage in enumerate(linkages):

            fig, ax = plt.subplots(4, 4, figsize=(30, 20))
            ax = ax.ravel()

            for k, distance_threshold in enumerate(distance_thresholds):
                # Create an instance of AgglomerativeClustering with desired parameters
                cluster = AgglomerativeClustering(
                    n_clusters=None,
                    affinity=affinity,
                    linkage=linkage,
                    distance_threshold=distance_threshold,
                )

                # Fit the model to the data
                cluster.fit(X)

                # Get the cluster labels for each data point
                labels = cluster.labels_

                score = 0
                # Calculate silhouette score
                if len(np.unique(labels.T)) > 1:
                    score = silhouette_score(X, labels)
                    sil_score.append(score)

                if score > best_score:
                    best_score = score
                    best_params = (affinity, linkage, distance_threshold)

                ax[k].scatter(X[:, 0], X[:, 1], c=labels, cmap="rainbow")
                ax[k].set_title(
                    f"Distance Threshold: {distance_threshold:.2f}\nSilhouette Score: {score:.2f}"
                )

                ax[k].set_xlabel("sepal_length")
                ax[k].set_ylabel("sepal_width")

                if np.any(labels.T):
                    # get centroids
                    clf = NearestCentroid()
                    # print(HClusteringY)
                    clf.fit(X, labels)
                    ax[k].scatter(
                        clf.centroids_[:, 0],
                        clf.centroids_[:, 1],
                        c="black",
                        s=100,
                        alpha=0.5,
                        marker="x",
                    )

            fig.suptitle(
                f"Agglomerative clustering with Affinity: {affinity} and Linkage: {linkage}"
            )

            fig.tight_layout()
            plt.show()
    return best_params, sil_score


best_params, sil_score = aggro_hierarchal_iris(0.1, 5)

# create silhouette graph
plt.figure()
plt.plot(range(len(sil_score)), sil_score)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(
    f"Best parameters: Affinity={best_params[0]}, Linkage={best_params[1]}, Distance Threshold={best_params[2]}"
)

### DBScan


In [ ]:
iris = load_iris()
X = iris.data
col = ["sepal length", "sepal width", "petal length", "petal width"]


def DBSCAN_iris(x, y):
    eps_values = np.linspace(0.1, 3.0, num=30)
    min_samples_values = np.linspace(5, 25, num=30)

    fig, axs = plt.subplots(6, 5, figsize=(35, 35))

    x_label = col[x]
    y_label = col[y]

    best_silhouette_score = -1
    best_eps = None
    best_min_samples = None
    sil_scores = []

    ax = axs.ravel()

    for i in range(len(eps_values)):
        eps = eps_values[i]
        min_samples = min_samples_values[i]

        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        labels = db.labels_

        ax[i].scatter(X[:, x], X[:, y], c=labels, cmap="gist_rainbow")
        ax[i].set_title(f"eps={eps:.2f}, min_samples={min_samples}")
        ax[i].set_xlabel(f"{x_label}")
        ax[i].set_ylabel(f"{y_label}")

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters > 1:
            silhouette_avg = silhouette_score(X, labels)
            sil_scores.append(silhouette_avg)

            if silhouette_avg > best_silhouette_score:
                best_silhouette_score = silhouette_avg
                best_eps = eps
                best_min_samples = min_samples

    fig.suptitle(f"DB SCAN of {x_label} versus {y_label}", y=1.01)
    fig.tight_layout()
    return best_eps, best_min_samples, sil_scores


best_eps, best_min_samples, sil_scores = DBSCAN_iris(0, 1)
plt.figure(figsize=(8, 5))
plt.plot(range(len(sil_scores)), sil_scores)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(f"Best eps: {best_eps:.2f}")
print(f"Best min samples: {best_min_samples}")

In [ ]:
db = DBSCAN(eps=0.99, min_samples=11.15).fit(X)
labels = db.labels_
plt.scatter(X[:, 0], X[:, 1], c=labels)
plt.title(f"eps=0.99, min_samples=11.15")


### GMM clustering

- Use GaussianMixture function to cluster the above data
- In GMM change the covariance_type and check the difference in the resulting proabability fit
- Use a 2D contour plot to plot the resulting distribution (the components of the GMM) as well as the total Gaussian mixture


In [ ]:
iris = load_iris()
X = iris.data[:, :2]  # we only take the first two features.


def GMM_iris(X):
    n_components_range = range(1, 21)
    covariance_types = ["spherical", "tied", "diag", "full"]

    for index, covariance_type in enumerate(covariance_types):
        plt.figure(figsize=(30, 18))
        for n_components in n_components_range:
            gmm = GaussianMixture(
                n_components=n_components, covariance_type=covariance_type
            )
            gmm.fit(X)

            plt.subplot(4, 5, n_components)

            x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
            y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

            xx, yy = np.meshgrid(np.linspace(x_min, x_max), np.linspace(y_min, y_max))
            Z = gmm.score_samples(np.c_[xx.ravel(), yy.ravel()])

            Z = Z.reshape(xx.shape)
            plt.contour(xx, yy, Z)

            plt.scatter(X[:, 0], X[:, 1], c=gmm.predict(X), cmap="gist_rainbow")
            plt.title(f"n_components={n_components}")

        plt.suptitle(f"GMM with {covariance_type} covariance")
        plt.tight_layout()
        plt.show()


GMM_iris(X)

## Repeat all steps after normalizing

### K-means


In [ ]:
iris = load_iris()
X = iris.data
X = prep.normalize(X)

kmeans_iris(0, 1)
kmeans_iris(2, 3)


# create elbow graph
plt.figure()
plt.plot(range(1, 11), distortion)
plt.title("The elbow method")
plt.xlabel("Number of clusters")
plt.ylabel("SUm of squares distance")  # within cluster sum of squares
plt.show()

# create and plot sil graph
plt.figure()
plt.plot(range(2, 11), sil_scores)
plt.xlabel("k")
plt.ylabel("Silhouette Score")
plt.show()


### Hierarchial Clustering


In [ ]:
iris = load_iris()
X = iris.data
X = prep.normalize(X)

best_params, sil_score = aggro_hierarchal_iris(0.01, 1)

# create silhouette graph
plt.figure()
plt.plot(range(len(sil_score)), sil_score)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(
    f"Best parameters: Affinity={best_params[0]}, Linkage={best_params[1]}, Distance Threshold={best_params[2]}"
)

### DBSCAN


In [ ]:
iris = load_iris()
X = iris.data
X = prep.normalize(X)


best_eps_petal, best_min_samples_petal, sil_score_petal = DBSCAN_iris(0, 1)
best_eps_sepal, best_min_samples_sepal, sil_score_sepal = DBSCAN_iris(2, 3)

# create silhouette graph
plt.figure()
plt.plot(range(len(sil_score_petal)), sil_score_petal)
plt.title("silhouette score petals")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

# create silhouette graph
plt.figure()
plt.plot(range(len(sil_score_sepal)), sil_score_sepal)
plt.title("silhouette score sepals")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(
    f"Best parameters for petals: eps={best_eps_petal}, min_samples={best_min_samples_petal}"
)
print(
    f"Best parameters for sepals: eps={best_eps_sepal}, min_samples={best_min_samples_sepal}"
)

plt.show()

### GMM clustering


In [ ]:
iris = load_iris()
X = iris.data[:, :2]  # we only take the first two features.
X = prep.normalize(X)


def GMM_iris(X, label_flag):
    n_components_range = range(1, 21)
    covariance_types = ["spherical", "tied", "diag", "full"]

    for index, covariance_type in enumerate(covariance_types):
        plt.figure(figsize=(30, 18))
        for n_components in n_components_range:
            gmm = GaussianMixture(
                n_components=n_components, covariance_type=covariance_type
            )
            gmm.fit(X)

            plt.subplot(4, 5, n_components)

            x_min, x_max = X[:, 0].min(), X[:, 0].max()
            y_min, y_max = X[:, 1].min(), X[:, 1].max()

            xx, yy = np.meshgrid(np.linspace(x_min, x_max), np.linspace(y_min, y_max))
            Z = gmm.score_samples(np.c_[xx.ravel(), yy.ravel()])

            Z = Z.reshape(xx.shape)
            plt.contour(xx, yy, Z)

            plt.scatter(X[:, 0], X[:, 1], c=gmm.predict(X), cmap="gist_rainbow")
            plt.title(f"n_components={n_components}")

            if label_flag == 0:
                plt.xlabel("petal_length")
                plt.ylabel("petal_width")
            else:
                plt.xlabel("sepal_length")
                plt.ylabel("sepal_width")

        plt.suptitle(
            f'GMM with {covariance_type} covariance for {"petals" if label_flag==0 else "Sepals"}'
        )
        plt.tight_layout()
        plt.show()


GMM_iris(X, 0)

X = iris.data[:, 2:4]  # we take the second two features.
X = prep.normalize(X)

print("Now for sepal length and width")

GMM_iris(X, 1)

- Repeat all the above clustering approaches and steps on the above data
- Normalize the data then repeat all the above steps
- Compare between the different clustering approaches


## Customer dataset

Repeat all the above on the customer data set


In [ ]:
from plotnine import *
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_blobs
import sklearn.preprocessing as prep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# StandardScaler is a function to normalize the data
# You may also check MinMaxScaler and MaxAbsScaler
#from sklearn.preprocessing import StandardScaler


# hierarchical clustering


# for cdist in distortions graph

%matplotlib inline


In [ ]:
customer_df = pd.read_csv("Customer data.csv")
print(customer_df.info())
print(customer_df.head())
print(customer_df.keys())

### K means


In [ ]:
customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)

# df = prep.normalize(df,axis=0)
# df = pd.DataFrame(df)
# print(df)
# df.values[:,0] = df.values[:,0] +10
# print(df)


def kmeans_cust(x, y, df):
    fig, axs = plt.subplots(2, 5, figsize=(20, 8))
    axs = axs.ravel()

    sil_scores = []
    distortions = []

    x_label = customer_df.columns[x+1]
    y_label = customer_df.columns[y+1]

    for i in range(0, 10):
        kmeans = KMeans(n_clusters=i + 1)
        kmeans.fit(df)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_
        distortions.append(kmeans.inertia_)

        if i > 0:
            sil_score = silhouette_score(df, labels)
            sil_scores.append(sil_score)
            axs[i].set_title(f"k={i}, silhouette={sil_score:.2f}", y=1.01)
        else:
            axs[i].set_title(f"k={i+1}", y=1.01)

        # plot cluster and centers
        axs[i].scatter(df[:, x], df[:, y], c=labels, cmap="gist_rainbow")
        axs[i].scatter(centers[:, x], centers[:, y], marker="o", s=100, color="black")

        axs[i].set_xlabel(f"{x_label}")
        axs[i].set_ylabel(f"{y_label}")

    fig.tight_layout()
    fig.suptitle(f"K means clustering of {x_label} versus {y_label}", y=1.03)
    return sil_scores, distortions


sil_scores, distortions = kmeans_cust(2, 4, X)
for i in range(0, len(df.keys())):
    if i != 2:
        kmeans_cust(i, 2, X)

for i in range(0, len(df.keys())):
    if i != 4:
        kmeans_cust(i, 4, X)
    # kmeans_cust(7,5,X)
# kmeans_cust(1,7,X)


# create elbow graph
plt.figure()
plt.plot(range(len(distortions)), distortions)
plt.title("The elbow method")
plt.xlabel("Number of clusters")
plt.ylabel("SUm of squares distance")  # within cluster sum of squares
plt.show()

# create and plot sil graph
plt.figure()
plt.plot(range(len(sil_scores)), sil_scores)
plt.xlabel("k")
plt.ylabel("Silhouette Score")
plt.show()

### Hierarchal clustering


In [ ]:
# customer_df = pd.read_csv("Customer data.csv")
# df = customer_df.drop(customer_df.columns[0], axis=1)
# X = prep.normalize(df)

customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)


def aggro_hierarchal_cust(dist_min, dist_max, x, y, df_cust):
    affinities = ["euclidean", "manhattan", "cosine"]
    linkages = ["average", "single"]
    distance_thresholds = np.linspace(dist_min, dist_max, num=16)

    # fig, axs = plt.subplots(len(affinities) * len(linkages), len(distance_thresholds), figsize=(50, 25))

    best_score = -1
    best_params = []
    sil_score = []

    x_label = customer_df.columns[x+1]
    y_label = customer_df.columns[y+1]

    for i, affinity in enumerate(affinities):
        for j, linkage in enumerate(linkages):

            fig, ax = plt.subplots(4, 4, figsize=(30, 20))
            ax = ax.ravel()

            for k, distance_threshold in enumerate(distance_thresholds):
                # Create an instance of AgglomerativeClustering with desired parameters
                cluster = AgglomerativeClustering(
                    n_clusters=None,
                    affinity=affinity,
                    linkage=linkage,
                    distance_threshold=distance_threshold,
                )

                # Fit the model to the data
                cluster.fit(df_cust)

                # Get the cluster labels for each data point
                labels = cluster.labels_

                score = 0
                # Calculate silhouette score
                if len(np.unique(labels.T)) > 1:
                    score = silhouette_score(df, labels)
                    sil_score.append(score)

                if score > best_score:
                    best_score = score
                    best_params = (affinity, linkage, distance_threshold)

                x_min = df_cust[:, x].min()
                x_max = df_cust[:, x].max()

                ax[k].scatter(df_cust[:, x], df_cust[:, y], c=labels, cmap="rainbow")
                ax[k].set_title(
                    f"Distance Threshold: {distance_threshold:.5f}\nSilhouette Score: {score:.2f}"
                )

                ax[k].set_xlabel(f"{x_label}")
                ax[k].set_ylabel(f"{y_label}")

                ax[k].set_xlim((x_min, x_max))

                if np.any(labels.T):
                    # get centroids
                    clf = NearestCentroid()
                    # print(HClusteringY)
                    clf.fit(df_cust, labels)
                    ax[k].scatter(
                        clf.centroids_[:, 0],
                        clf.centroids_[:, 1],
                        c="black",
                        s=100,
                        alpha=0.5,
                        marker="x",
                    )

            fig.suptitle(
                f"Agglomerative clustering with Affinity: {affinity} and Linkage: {linkage}",y=1.01,fontsize=20)

            fig.tight_layout()
            plt.show()
    return best_params, sil_score


best_params, sil_score = aggro_hierarchal_cust(0.05,1.5,2,4,X)


# create silhouette graph
plt.figure(figsize=(15, 10))
# plt.plot(range(0, 16), sil_score[0:16], "bx-", label="euclidean average", color="r")
# plt.plot(range(16, 18), sil_score[16:18], "bx-", label="euclidean single", color="b")
# plt.plot(range(18, 34), sil_score[18:34], "bx-", label="manhattan average", color="g")
# plt.plot(range(34, 36), sil_score[34:36], "bx-", label="manhattan single", color="m")
plt.plot(range(len(sil_score)) ,sil_score)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
# plt.legend(loc="lower right")
plt.show()

print(f"Best parameters: Affinity={best_params[0]}, Linkage={best_params[1]}, Distance Threshold={best_params[2]}")

In [ ]:
customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)

neighb = NearestNeighbors(
    n_neighbors=5
)  # creating an object of the NearestNeighbors class
nbrs = neighb.fit(X)  # fitting the data to the object
distances, indices = nbrs.kneighbors(X)  # finding the nearest neighbours

distances = np.sort(distances, axis=0)  # sorting the distances
distances = distances[:, 1]  # taking the second column of the sorted distances
plt.rcParams["figure.figsize"] = (5, 3)  # setting the figure size
plt.plot(distances)  # plotting the distances
plt.show()  # showing the plot

### DB SCAN


In [ ]:
customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)


def DBSCAN_cust(x, y, inv, cust_df):
    eps_values = np.linspace(0.01,2, num=30)
    if inv == False:
        min_samples_values = np.linspace(5, 25, num=30)
    else:
        min_samples_values = np.linspace(25, 5, num=30)

    fig, axs = plt.subplots(6, 5, figsize=(35, 35))

    x_label = df.columns[x]
    y_label = df.columns[y]

    best_silhouette_score = -1
    best_eps = None
    best_min_samples = None
    sil_scores = []

    ax = axs.ravel()
    for i in range(len(eps_values)):
        eps = eps_values[i]
        min_samples = min_samples_values[i]

        db = DBSCAN(eps=eps, min_samples=min_samples).fit(cust_df)
        labels = db.labels_

        ax[i].scatter(cust_df[:, x], cust_df[:, y], c=labels, cmap="gist_rainbow")
        ax[i].set_title(f'eps={eps:.2f}, min_samples={min_samples:.2f}')
        ax[i].set_xlabel(f"{x_label}")
        ax[i].set_ylabel(f"{y_label}")

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters > 1:
            silhouette_avg = silhouette_score(cust_df, labels)
            sil_scores.append(silhouette_avg)

            if silhouette_avg > best_silhouette_score:
                best_silhouette_score = silhouette_avg
                best_eps = eps
                best_min_samples = min_samples

    fig.suptitle(f"DB SCAN of {x_label} versus {y_label}", y=1.01)
    fig.tight_layout()
    return best_eps, best_min_samples, sil_scores


# for i in range(1, len(customer_df.keys())):
#     if i != 3:
#         DBSCAN_cust(i, 3)

# for i in range(1, len(customer_df.keys())):
#     if i != 5:
#         DBSCAN_cust(i, 5)

best_eps, best_min_samples, sil_scores = DBSCAN_cust(2, 4, True, X)
DBSCAN_cust(3, 4, False, X)
DBSCAN_cust(6, 3, False, X)
DBSCAN_cust(2, 5, True, X)
plt.figure(figsize=(8, 5))
plt.plot(range(len(sil_scores)), sil_scores)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(f"Best eps: {best_eps:.2f}")
print(f"Best min samples: {best_min_samples}")

### GMM clustering

In [ ]:
customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)


def GMM_cust(X):
    n_components_range = range(1, 21)
    covariance_types = ["spherical", "tied", "diag", "full"]

    for index, covariance_type in enumerate(covariance_types):
        plt.figure(figsize=(30, 18))
        for n_components in n_components_range:
            gmm = GaussianMixture(
                n_components=n_components, covariance_type=covariance_type
            )
            gmm.fit(X)

            plt.subplot(4, 5, n_components)

            x_min, x_max = X[:, 0].min(), X[:, 0].max()
            y_min, y_max = X[:, 1].min(), X[:, 1].max()


            xx, yy = np.meshgrid(np.linspace(x_min, x_max), np.linspace(y_min, y_max))
            # print(np.c_[xx.ravel(), yy.ravel()].shape)
            Z = gmm.score_samples(np.c_[xx.ravel(), yy.ravel()])

            Z = Z.reshape(xx.shape)
            plt.contour(xx, yy, Z)

            plt.scatter(X[:, 0], X[:, 1], c=gmm.predict(X), cmap="gist_rainbow")
            plt.title(f"n_components={n_components}")

        plt.suptitle(
            f'GMM with {covariance_type} covariance for '
        )
        plt.tight_layout()
        plt.show()


GMM_cust(X[:,2:5:2])
# print(X[:,2:5:2])

# GMM_cust(X)

## Extra requirements
- use PCA to reduce dimensionality
- Test different distance function other then Euclidean and study their effects

In [ ]:
from plotnine import *
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_blobs
import sklearn.preprocessing as prep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# StandardScaler is a function to normalize the data
# You may also check MinMaxScaler and MaxAbsScaler
#from sklearn.preprocessing import StandardScaler
# hierarchical clustering
# for cdist in distortions graph

%matplotlib inline
from sklearn.decomposition import PCA

In [ ]:
customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)


# PCA
pca = PCA(n_components=2)
reduced_X = pca.fit_transform(X)


sil_scores_kmeans, distortions = kmeans_cust(0, 1, reduced_X)

# create elbow graph
plt.figure()
plt.plot(range(len(distortions)), distortions)
plt.title("The elbow method")
plt.xlabel("Number of clusters")
plt.ylabel("SUm of squares distance")  # within cluster sum of squares
plt.show()

# create and plot sil graph
plt.figure()
plt.plot(range(len(sil_scores_kmeans)), sil_scores_kmeans)
plt.xlabel("k")
plt.ylabel("Silhouette Score")
plt.show()


best_params_aggro, sil_score_aggro = aggro_hierarchal_cust(0.05,1.5,0,1,reduced_X)

# create silhouette graph
plt.figure(figsize=(15, 10))
# plt.plot(range(0, 16), sil_score[0:16], "bx-", label="euclidean average", color="r")
# plt.plot(range(16, 18), sil_score[16:18], "bx-", label="euclidean single", color="b")
# plt.plot(range(18, 34), sil_score[18:34], "bx-", label="manhattan average", color="g")
# plt.plot(range(34, 36), sil_score[34:36], "bx-", label="manhattan single", color="m")
plt.plot(range(len(sil_score_aggro)) ,sil_score_aggro)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
# plt.legend(loc="lower right")
plt.show()

print(f"Best parameters: Affinity={best_params_aggro[0]}, Linkage={best_params_aggro[1]}, Distance Threshold={best_params_aggro[2]}")


best_eps, best_min_samples, sil_scores = DBSCAN_cust(0, 1, True,reduced_X)
plt.figure(figsize=(8, 5))
plt.plot(range(len(sil_scores)), sil_scores)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(f"Best eps: {best_eps:.2f}")
print(f"Best min samples: {best_min_samples}")

GMM_cust(reduced_X)


## Using Mahalanobis Distance

In [ ]:
from plotnine import *
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_blobs
import sklearn.preprocessing as prep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# StandardScaler is a function to normalize the data
# You may also check MinMaxScaler and MaxAbsScaler
#from sklearn.preprocessing import StandardScaler
# hierarchical clustering
# for cdist in distortions graph

%matplotlib inline
from sklearn.decomposition import PCA
from scipy.spatial.distance import mahalanobis

In [ ]:
def mahalanobis_distance(x, y):
    x = np.array(x)
    y = np.array(y)
    VI = np.linalg.inv(np.cov(x.T))
    return mahalanobis(x, y, VI)

### K means

In [ ]:
customer_df = pd.read_csv("Customer data.csv")
df = customer_df.drop(customer_df.columns[0], axis=1)
scaler = prep.MinMaxScaler()
normalized_data = scaler.fit_transform(df)
X_dataframe = pd.DataFrame(normalized_data)
X = pd.DataFrame.to_numpy(X_dataframe)


def DBSCAN_cust(x, y, inv, cust_df):
    eps_values = np.linspace(0.01,2, num=30)
    if inv == False:
        min_samples_values = np.linspace(5, 25, num=30)
    else:
        min_samples_values = np.linspace(25, 5, num=30)

    fig, axs = plt.subplots(6, 5, figsize=(35, 35))

    x_label = df.columns[x]
    y_label = df.columns[y]

    best_silhouette_score = -1
    best_eps = None
    best_min_samples = None
    sil_scores = []

    ax = axs.ravel()
    for i in range(len(eps_values)):
        eps = eps_values[i]
        min_samples = min_samples_values[i]

        # cov_X = X[:,x] 

        cov_X = np.stack((X[:,x],X[:,y]))

        metric_params={'V':np.cov(cov_X)}

        db = DBSCAN(eps=eps, min_samples=min_samples,metric="mahalanobis",metric_params=metric_params).fit(cust_df)
        labels = db.labels_

        ax[i].scatter(cust_df[:, x], cust_df[:, y], c=labels, cmap="gist_rainbow")
        ax[i].set_title(f'eps={eps:.2f}, min_samples={min_samples:.2f}')
        ax[i].set_xlabel(f"{x_label}")
        ax[i].set_ylabel(f"{y_label}")

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters > 1:
            silhouette_avg = silhouette_score(cust_df, labels)
            sil_scores.append(silhouette_avg)

            if silhouette_avg > best_silhouette_score:
                best_silhouette_score = silhouette_avg
                best_eps = eps
                best_min_samples = min_samples

    fig.suptitle(f"DB SCAN of {x_label} versus {y_label}", y=1.01)
    fig.tight_layout()
    return best_eps, best_min_samples, sil_scores


# for i in range(1, len(customer_df.keys())):
#     if i != 3:
#         DBSCAN_cust(i, 3)

# for i in range(1, len(customer_df.keys())):
#     if i != 5:
#         DBSCAN_cust(i, 5)

best_eps, best_min_samples, sil_scores = DBSCAN_cust(2, 4, True, X)
DBSCAN_cust(3, 4, False, X)
DBSCAN_cust(6, 3, False, X)
DBSCAN_cust(2, 5, True, X)
plt.figure(figsize=(8, 5))
plt.plot(range(len(sil_scores)), sil_scores)
plt.title("silhouette score")
plt.xlabel("different parameters")
plt.ylabel("score")  # within cluster sum of squares
plt.show()

print(f"Best eps: {best_eps:.2f}")
print(f"Best min samples: {best_min_samples}")